In [1]:
import random
import numpy as np
import tensorflow as tf
from IPython.display import clear_output
k = tf.keras

In [2]:
k.backend.clear_session()

In [3]:
Qmodel = k.Sequential([
    k.layers.Input(shape=[4]),
    k.layers.Dense(units=8,activation='relu'),
    k.layers.Dense(units=8,activation='relu'),
    k.layers.Dense(units=8,activation='relu'),
    k.layers.Dense(units=2)
])

In [ ]:
QtargetModel = k.models.clone_model(Qmodel)
QtargetModel.set_weights(Qmodel.get_weights())

In [ ]:
Qmodel.compile('adam',loss=k.losses.huber)

In [ ]:
import gym
env = gym.make('CartPole-v1')

In [ ]:
def best_action(state):
    Qsa = Qmodel(state)[0].numpy()
    return np.argmax(Qsa)

def best_Q_value(state):
    Qsa = QtargetModel(state)[0].numpy()
    return Qsa[action]

In [ ]:
EPSILON = 1
MIN_EPSILON = 0.05
DECAY = 0.005
DISCOUNT = 0.9

MAX_HISTORY = 1000
replay_memory = []

BATCH_SIZE = 64

In [ ]:
def train():
    memories = random.choices(replay_memory, k=BATCH_SIZE)
    batch = []
    target = []
    for sample in memories:
        tmp = Qmodel(sample['state'])[0].numpy()
        if sample['done']:
            tmp[sample['action']] = sample['reward']
            target.append(tmp)
        else:
            tmp[sample['action']] = sample['reward'] + DISCOUNT * best_Q_value(sample['next_state'])
            target.append(tmp)
            
        batch.append(sample['state'][0])
    
    target = np.array(target)
    batch = np.array(batch)
    h = Qmodel.fit(batch, target, verbose=0)
    
    return h.history['loss'][0]

In [ ]:
history = {
    'rewards':[],
    'losses':[],
}

In [ ]:
MAX_GAME = 5000
game = 0
step = 0
MAX_ITER = 200
good_game = 0
while game < MAX_GAME:
    game += 1
    
    state = env.reset()
    state = np.array([state])
    
    done = False
    cumulative_reward = 0
    for i in range(MAX_ITER):
        if np.random.uniform() < EPSILON:
            action = env.action_space.sample()
        else:
            action = best_action(state)
        
        next_state, reward, done, info = env.step(action)
        next_state = np.array([next_state])
        
        cumulative_reward += reward
        
        replay_memory.append({
            'state': state,
            'action': action,
            'reward': reward,
            'next_state': next_state,
            'done': done,
        })
        
        if len(replay_memory) > MAX_HISTORY:
            replay_memory.pop(0)
            
        step += 1
        if done:
            break
        
        state = next_state
    
    if EPSILON > MIN_EPSILON:
        EPSILON = EPSILON * np.exp(-DECAY)
        
    loss = train()
    if game % 50 == 0:
        QtargetModel.set_weights(Qmodel.get_weights())
        
    
    history['rewards'].append(cumulative_reward)
    history['losses'].append(loss)
    
    print(f'Game {game}: {cumulative_reward} --- loss = {loss}')
    
    if cumulative_reward > 170:
        good_game+=1
    else:
        good_game=0
        
    if good_game >= 100:
        break
    
    clear_output(wait=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history['rewards'])

In [ ]:
plt.plot(history['losses'])

In [ ]:
state = env.reset()
state = np.array([state])
done = False
while not done:
    action = best_action(state)
    next_state, reward, done, info = env.step(action)
    next_state = np.array([next_state])
    env.render()
    
    state = next_state

env.close()